In [1]:
import anywidget
import ipywidgets as widgets
from IPython.display import display
import requests
import traitlets
import pandas as pd
from urllib.parse import urlencode

In [2]:
data = {
    'company_name': ['Total', 'Nestle', 'LVMH Moët Hennessy - Louis Vuitton SE', 'Unilever', 'GlaxoSmithKline plc', 'Royal Dutch Shell plc', 'Eni S.p.A.', 'Volkswagen AG', 'Bayer AG', 'Airbus SE'],
    'jurisdiction_code': ['FR', 'CH', 'FR', 'GB', 'GB', 'GB', 'IT', 'DE', 'DE', 'FR'],
    'reporting_date': ['2015-03-31', '2018-12-31', '2013-06-30', '2017-09-30', '2011-12-31', '2014-03-31', '2019-06-30', '2016-12-31', '2012-09-30', '2020-03-31']
}

df = pd.DataFrame(data)

# Display the first 10 rows
print(df.head(10))


                            company_name jurisdiction_code reporting_date
0                                  Total                FR     2015-03-31
1                                 Nestle                CH     2018-12-31
2  LVMH Moët Hennessy - Louis Vuitton SE                FR     2013-06-30
3                               Unilever                GB     2017-09-30
4                    GlaxoSmithKline plc                GB     2011-12-31
5                  Royal Dutch Shell plc                GB     2014-03-31
6                             Eni S.p.A.                IT     2019-06-30
7                          Volkswagen AG                DE     2016-12-31
8                               Bayer AG                DE     2012-09-30
9                              Airbus SE                FR     2020-03-31


In [7]:
class ReconciliationWidget(anywidget.AnyWidget):
    _esm = """
    export function render(view) {
        const container = document.createElement('div');
        view.el.appendChild(container);
    
        function updateWidget(data) {
            container.innerHTML = '';
            if (data && data.length) {
                data.forEach((item, index) => {
                    const row = document.createElement('div');
                    row.innerHTML = `
                        <input type="radio" name="match" value="${index}" id="match_${index}">
                        <label for="match_${index}">${item.name} (Score: ${item.score}, ID: ${item.id})</label>
                    `;
                    container.appendChild(row);
                });
            } else {
                container.innerHTML = '<p>No results found.</p>';
            }
        }
    
        view.model.on('change:results', () => {
            updateWidget(view.model.get('results'));
        });
    
        container.addEventListener('change', (event) => {
            if (event.target.name === 'match') {
                view.model.set('selected', parseInt(event.target.value));
                view.model.save_changes();
            }
        });
    }
    """

    results = traitlets.List().tag(sync=True)
    selected = traitlets.Int(-1).tag(sync=True)
    current_index = traitlets.Int(0).tag(sync=True)
    status = traitlets.Unicode('Ready').tag(sync=True)
    reconciliation_info = traitlets.Unicode('').tag(sync=True)

    def __init__(self, df, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.df = df
        self.next_button = widgets.Button(description="Next")
        self.next_button.on_click(self.on_next_click)
        self.company_label = widgets.Label()
        self.status_label = widgets.Label()
        self.reconciliation_label = widgets.Label()
        self.output = widgets.Output()
        self.create_column_dropdowns()
        self.update_company_label()
        self.df['reconciled_id'] = ''

        self.observe(self.on_selection, names='selected')
        self.observe(self.update_status_label, names='status')
        self.observe(self.update_reconciliation_label, names='reconciliation_info')

    def create_column_dropdowns(self):
        columns = list(self.df.columns)
        self.query_dropdown = widgets.Dropdown(options=columns, description='Query:')
        self.jurisdiction_dropdown = widgets.Dropdown(options=columns, description='Jurisdiction:')
        self.date_dropdown = widgets.Dropdown(options=columns, description='Date:')

    def update_company_label(self):
        if self.current_index < len(self.df):
            company = self.df.iloc[self.current_index]
            self.company_label.value = f"Current: {company[self.query_dropdown.value]} ({company[self.jurisdiction_dropdown.value]})"
        else:
            self.company_label.value = "All companies processed"
            self.next_button.disabled = True

    def update_status_label(self, change):
        self.status_label.value = f"Status: {change['new']}"

    def update_reconciliation_label(self, change):
        self.reconciliation_label.value = change['new']

    def on_next_click(self, b):
        with self.output:
            self.output.clear_output()
            if self.current_index < len(self.df):
                self.status = 'Processing'
                self.reconciliation_info = ''  # Clear previous reconciliation info
                self.results = []  # Clear previous results
                self.process_current_company()
            else:
                print("All companies have been processed.")
                self.status = 'Completed'
                self.reconciliation_info = ''

    def process_current_company(self):
        company = self.df.iloc[self.current_index]
        if all(col.value in self.df.columns for col in [self.query_dropdown, self.jurisdiction_dropdown, self.date_dropdown]):
            query = company[self.query_dropdown.value]
            jurisdiction = company[self.jurisdiction_dropdown.value]
            date = company[self.date_dropdown.value]

            self.reconciliation_info = f"Reconciling: {query} ({jurisdiction}) as of {date}"

            try:
                url = f"https://opencorporates.com/reconcile?query={query}&jurisdiction_code={jurisdiction}&date={date}"
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                data = response.json()
                self.results = data.get('result', [])[:5]
                self.reconciliation_info += f"\nFound {len(self.results)} potential matches."
                self.status = 'Awaiting selection' if self.results else 'No matches found'
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                self.results = []
                self.status = 'Error occurred'
                self.reconciliation_info += "\nAn error occurred while fetching results."

            self.selected = -1
        else:
            print("Invalid column selection in dropdowns.")
            self.status = 'Invalid selection'
            self.reconciliation_info = "Invalid column selection"

    def on_selection(self, change):
        if change['new'] != -1:
            if self.results:
                selected_id = self.results[change['new']]['id']
                self.df.at[self.current_index, 'reconciled_id'] = selected_id
            else:
                self.df.at[self.current_index, 'reconciled_id'] = 'No match'
            
            self.current_index += 1
            self.update_company_label()
            self.results = []  # Clear results for the next company
            self.selected = -1  # Reset selection
            self.status = 'Ready for next company'
            self.reconciliation_info = ''  # Clear reconciliation info

    def display(self):
        layout = widgets.VBox([
            self.company_label,
            self.query_dropdown,
            self.jurisdiction_dropdown,
            self.date_dropdown,
            self.next_button,
            self.status_label,
            self.reconciliation_label,
            self.output,
            self
        ])
        display(layout)

In [8]:
widget = ReconciliationWidget(df)
widget.display()

In [ ]:
df